# TP2 - Conway’s Game of Life
Grupo 05

    Eduardo André Silva Cunha A98980
    Gonçalo Emanuel Ferreira Magalhães A100084

# Problema

2. O Conway’s Game of Life é um exemplo conhecido de um autómato celular . Aqui vamos modificar as regras do autómato de forma a usar um espaço de estados finito
    1. O espaço de estados é definido por uma grelha de células booleanas (morta=0/viva=1) de dimensão $\,N\times N\,$ (com $N>3$) identificadas por índices $\,(i,j)\in \{1..N\}$.  Estas $\;N^2\;$ células são aqui referidas como “normais”. 
    2. Inicialmente todas as células normais estão mortas excepto  as células $\,i,j \leq 3\,$ que estão vivas. Um estado onde todas as células normais estão mortas é um “estado de erro”.
    3. Adicionalmente existem $\,2\,N+1\,$ “células da borda” que correspondem a um dos índices, $i$ ou $j$, ser zero. As células da borda têm valores constantes que, no estado inicial, são gerados aleatoriamente com uma probabilidade $\,1/2\,$ de estarem vivas.
    4. As células normais o autómato modificam o estado de acordo com a regra “B3/S23”: i.e. a célula nasce (passa de $0$ a $1$) se tem exatamente 3 vizinhos vivos e sobrevive (mantém-se viva) se o número de vizinhos vivos é 2 ou 3, caso contrário morre ou continua morta.

Valores definidos para o problema e Inputs

In [1]:
from z3 import *
import random

global N # Tamanho da matriz
N = 7

Função que gera aleatóriamente um valor (0 ou 1) com 50% de chance de sair cada.

In [2]:
def gera_aleatorio():
    return random.choice([0, 1]) #gera 0 ou 1, 50% cada

Posições da matriz

In [3]:
pos = [f'({i},{j})' for i in range(N) for j in range(N)]

Função que dá as posições que não são bordas e que i,j > 3

In [4]:
def filtra_posicoes():
    posicoes_filtradas = []

    for i in range(N):
        for j in range(N):
            if i > 0 and j > 0:
                if not (i <= 3 and j <= 3):
                    #posicoes_filtradas.append(f'({i},{j})')
                    posicoes_filtradas.append((i,j))

    return posicoes_filtradas


global restantes_pos
restantes_pos = filtra_posicoes()


Função que declara as variáveis de estado num dicionário permitindo assim aceder às mesmas pelo nome.

In [5]:
def declare(k):
    state = {}
    state['pc'] = Int('pc'+str(k)) # +str(k) conveção

    for i in range(N):
        for j in range(N):
            state[f'({i},{j})'] = Int('('+str(i)+','+str(j)+')') # "(i,j)"
    return state

Função "init" garante que o estado inicial respeita as seguintes restrições:
- Todos as posições a 0, excepto se tiver nos indices onde i, j <= 3.
- Todas as posições da borda têm 50% de chance de ser 1.
- Garante que pc == 0.


In [6]:
def init(state):
    return And(
        state['pc'] == 0,
        And([And([state[f'({i},{j})'] == 1 for i in range(0, 4)]) for j in range(0, 4)]),  # Todas as posições (i, j) onde i, j <= 3 são 1
        And([state[f'(0,{i})'] == gera_aleatorio() for i in range(4, N)]),  # Inicializa a primeira coluna com valores aleatórios
        And([state[f'({i},0)'] == gera_aleatorio() for i in range(4, N)]),  # Inicializa a primeira linha com valores aleatórios
        And([state[f'({i},{j})'] == 0 for (i,j) in restantes_pos])  # Restantes posições a 0
    )

Função que conta o número de vizinhos de uma celula dado o seu estado.

In [7]:
def vizinhos_vivos(cel, state):
    # Extrai as coordenadas x e y da string cel no formato "(a,b)"
    x = int(cel[1])
    y = int(cel[3])
    
    # movimentos: direita, acima, esquerda e abaixo
    moves = [(x+1, y), (x, y+1), (x-1, y), (x, y-1)]
    
    # Contador
    vizinhos_vivos_count = 0

    for move in moves:
        x_move, y_move = move

        if 0 <= x_move < N and 0 <= y_move < N: # Para não aceder a locais fora da matriz
            # Verifica se o estado da célula vizinha é igual a 1 (viva)
            if state[f'({x_move},{y_move})'] == 1:
                vizinhos_vivos_count += 1

    return vizinhos_vivos_count

Função que verifica se uma célula pode nascer, ou seja se possui 3 vizinhos vivos

In [8]:
#Caso a célula possua 3 vizinhos vivos a célula passa de 0 para 1.
def nasce(curr, prox):
    # lista que vai conter as celulas que ficarão vivas no próximo estado
    nasce_conditions = []

    # Itera sobre todas as posições
    for v in pos:
        # Verifica se a célula atual (curr[v]) está morta (0)
        if curr[v] == 0:
            # Verifica se o número de vizinhos vivos é igual a 3
            if vizinhos_vivos(v, curr) == 3:
                nasce_conditions.append(prox[v] == 1)

    # And para todas as condições de nascimento no prox estado
    return And(nasce_conditions)

Função que verifica se uma célula sobrevive ou não, no caso necessita de 2 ou 3 vizinhos vivos para se manter viva

In [9]:
# caso a célula possua 2 ou 3 vizinhos vivos, caso contrário morre
def sobrevive(curr, prox):
    # lista que vai conter as celulas que ficarão vivas no próximo estado
    sobrevive_conditions = []

    # Itera sobre todas as posições
    for v in pos:
        # Verifica se a célula atual (curr[v]) está viva (1)
        if curr[v] == 1:
            # Verifica se o número de vizinhos vivos é igual a 2 ou 3
            if vizinhos_vivos(v, curr) == 3 or vizinhos_vivos(v, curr) == 2:
                sobrevive_conditions.append(prox[v] == 1)

    # And para todas as condições de sobrevivência no prox estado
    return And(sobrevive_conditions)

Função que verificam se a celula deve ser morta no próximo estado, no caso isso acontece se tiver menos de 2 ou mais de 3 vizinhos vivos.

In [10]:
#Caso a célula não possua 2 nem 3 vizinhos vivos.
def morre(curr, prox):
    # lista que vai conter as celulas que ficarão vivas no próximo estado
    morre_conditions = []

    # Itera sobre todas as posições 
    for v in pos:
        # Verifica se a célula atual (curr[v]) está viva (1)
        if curr[v] == 1:
            # Verifica se o número de vizinhos vivos é maior ou igual a 4 ou menor q 2 -- GE (greater or equal)
            if UGE(vizinhos_vivos(v, curr), 4) or Not(UGE(vizinhos_vivos(v, curr), 2)):
                morre_conditions.append(prox[v] == 0)

    # And para todas as condições de morte no prox estado
    return And(morre_conditions)

Função que verifica se uma célula permanece morta no proximo estado, ou seja se já se encontrar morta e possuir menos de 2 ou mais de 3 vizinhos vivos

In [11]:
# Caso a célula continue sem possuir nem 2, nem 3 vizinhos, então permanece no seu estado morta.
def continua_morta(curr, prox):
    # lista que vai conter as celulas que ficarão vivas no próximo estado
    continua_morta_conditions = []

    # Itera sobre todas as posições
    for v in pos:
        # Verifica se a célula atual (curr[v]) está morta (0)
        if curr[v] == 0:
            # Verifica se o número de vizinhos vivos é maior ou igual a 4 ou menor que 2 -- GE(greater or equal)
            if UGE(vizinhos_vivos(v, curr), 4) or Not(UGE(vizinhos_vivos(v, curr), 2)):
                continua_morta_conditions.append(prox[v] == 0)

    # And para todas as condições de continuar morta no prox estado
    return And(continua_morta_conditions)

Função que dados dois possíveis estados do programa, testa se é possível transitar de um estado para o outro.

In [12]:
def trans(curr,prox):
    # Avança de estado
    t0 = And(
             prox['pc'] == curr['pc']+1, # avanca o pc
             nasce(curr,prox), # vê que celulas nascem
             sobrevive(curr,prox), # vê que celulas sobrevivem
             morre(curr,prox), # vê que celulas morrem
             continua_morta(curr,prox) # vê que celulas continuam mortas
    )

    # Mantem se igual
    t1 = And(
              prox['pc'] == curr['pc'],
              prox == curr     
         )
    
    return Or(t0,t1)

Função que gera um possível traço de execução do programa para k passos.

In [13]:
def gera_traco(declare,init,trans,k):
    s = Solver()

    trace = [declare(i) for i in range(k)]

    s.add(init(trace[0]))
    
    for i in range(k-1):
        s.add(trans(trace[i], trace[i+1]))
        
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print("Passo:", i)
            for v in trace[i]:
                print(f"{v} = {m[trace[i][v]]}")
            print("--------//--------")
        
    else:
        print("Não foi possível gerar o traço.\n")


gera_traco(declare,init,trans,10)

Passo: 0
pc = 0
(0,0) = 1
(0,1) = 1
(0,2) = 1
(0,3) = 1
(0,4) = 1
(0,5) = 1
(0,6) = 1
(1,0) = 1
(1,1) = 1
(1,2) = 1
(1,3) = 1
(1,4) = 0
(1,5) = 0
(1,6) = 0
(2,0) = 1
(2,1) = 1
(2,2) = 1
(2,3) = 1
(2,4) = 0
(2,5) = 0
(2,6) = 0
(3,0) = 1
(3,1) = 1
(3,2) = 1
(3,3) = 1
(3,4) = 0
(3,5) = 0
(3,6) = 0
(4,0) = 0
(4,1) = 0
(4,2) = 0
(4,3) = 0
(4,4) = 0
(4,5) = 0
(4,6) = 0
(5,0) = 1
(5,1) = 0
(5,2) = 0
(5,3) = 0
(5,4) = 0
(5,5) = 0
(5,6) = 0
(6,0) = 1
(6,1) = 0
(6,2) = 0
(6,3) = 0
(6,4) = 0
(6,5) = 0
(6,6) = 0
--------//--------
Passo: 1
pc = 1
(0,0) = 1
(0,1) = 1
(0,2) = 1
(0,3) = 1
(0,4) = 1
(0,5) = 1
(0,6) = 1
(1,0) = 1
(1,1) = 1
(1,2) = 1
(1,3) = 1
(1,4) = 0
(1,5) = 0
(1,6) = 0
(2,0) = 1
(2,1) = 1
(2,2) = 1
(2,3) = 1
(2,4) = 0
(2,5) = 0
(2,6) = 0
(3,0) = 1
(3,1) = 1
(3,2) = 1
(3,3) = 1
(3,4) = 0
(3,5) = 0
(3,6) = 0
(4,0) = 0
(4,1) = 0
(4,2) = 0
(4,3) = 0
(4,4) = 0
(4,5) = 0
(4,6) = 0
(5,0) = 1
(5,1) = 0
(5,2) = 0
(5,3) = 0
(5,4) = 0
(5,5) = 0
(5,6) = 0
(6,0) = 1
(6,1) = 0
(6,2) = 0
(6,3) = 0

Primeira verificação: Nunca se alcança um estado de erro

In [14]:
# Estado de erro -> todas as celulas estarem mortas
def testa_prop_1(declare, init, trans, inv, K):
    s = Solver()
    trace = [declare(i) for i in range(K)]

    s.add(init(trace[0]))

    for i in range(K):
        s.add(inv(trace[i]))

    for i in range(K - 1):
        # Adicione as transições
        s.add(trans(trace[i], trace[i+1]))

    # Sendo o estado de erro, todas as celulas estarem mortas
    prop_violada = (And([inv(trace[i]) for i in range(K)]))
    
    # Verifica se a propriedade é violada em algum estado do traço
    if s.check([prop_violada]) == sat: # Se satisfizer é porque em algum estado todas estão mortas
        print("Propriedade violada num estado do traço.")
    else:
        print("Propriedade mantida em todos os estados do traço.")


# Função que verifica se num estado todas as células estão mortas
def todas_mortas(state):
    return (And([state[celula] == 0 for celula in pos]))


testa_prop_1(declare, init, trans, todas_mortas, 10)


Propriedade mantida em todos os estados do traço.


Segunda Verificação: Nenhuma celula está permanentemente morta, ou viva.

In [15]:
#Nenhuma célula normal está permanentemente morta ou permanentemente viva.
def testa_prop_2(declare, init, trans, inv, K):
    s = Solver()
    trace = [declare(i) for i in range(K)]

    s.add(init(trace[0]))

    # adiciona a restrição de uma celula estar permanentemente viva ou morta
    # passa o trace completo porque a função verifica todos os estados para cada celula
    s.add(inv(trace))

    for i in range(K - 1):
        s.add(trans(trace[i], trace[i+1]))

    if s.check() == sat:
        print("Propriedade violada num estado do traço.")
    else:
        print("Propriedade mantida em todos os estados do traço.")


def nenhuma_celula_permanente(states):
    # para cada posicao
    for i in range(N):
        for j in range(N):
            # em cada estado
            for state in states:
                is_permanentemente_viva = True
                is_permanentemente_morta = True
                if state[f'({i},{j})'] == 1:
                    is_permanentemente_morta = False
                elif state[f'({i},{j})'] == 0:
                    is_permanentemente_viva = False
                    
                # se alguma for true, retorna False
                if is_permanentemente_viva or is_permanentemente_morta:
                    return False
    return True


testa_prop_2(declare, init, trans, nenhuma_celula_permanente, 10)


Propriedade mantida em todos os estados do traço.
